In [1]:
import numpy as np
import pandas as pd

In [2]:
eps = np.finfo(float).eps

In [3]:
df = pd.read_csv('./data.csv',index_col = 'RID')
df

,age,income,student,credit rating,buys computer
RID,,,,,
1,young,high,no,excellent,yes
2,young,high,no,good,yes
3,middle-aged,high,no,excellent,no
4,senior,medium,no,excellent,no
5,senior,low,yes,excellent,no
6,senior,low,yes,good,yes
7,middle-aged,low,yes,good,yes
8,young,medium,no,excellent,yes
9,young,low,yes,excellent,yes


In [8]:
def find_entropy(df):
    e = 0
    target = df.keys()[-1]
    for outcome in np.unique(df[target]):
        f = len(df[target][df[target]==outcome])/ len(df)
        e+=f*np.log2(f)
    return abs(e)

In [22]:
def find_entropy_att(df,attribute):
    target = df.keys()[-1]
    res = 0
    for var in np.unique(df[attribute]):
        e = 0
        for t in np.unique(df[target]):
            num =len(df[attribute][df[attribute]==var][df[target]==t])
            den =len(df[attribute][df[attribute]==var])
            f = num/(den+eps)
            e+=f*np.log2(f+eps)
        f2=den/len(df)
        res+=e*f2
    return abs(res)

In [23]:
def find_winner(df,att_list):
    IG = []
    for a in att_list:
        IG.append(find_entropy(df) - find_entropy_att(df,a))
    return att_list[np.argmax(IG)]
                  

In [24]:
def getsubtable(df,node,value):
    return df[df[node]==value].reset_index(drop =True)
    

In [25]:
def buildtree(df,att_list,tree=None):
    target=df.keys()[-1]
    if (len(att_list)==0):
        return df[target].mode()[0]

    node = find_winner(df,att_list)
    att_list.remove(node)

    if tree is None:
        tree={}
        tree[node]={}
    for value in np.unique(df[node]):
        subtable=getsubtable(df,node,value)
        feat_val, counts = np.unique(subtable[target],return_counts=True)
        if len(counts)==1:
            tree[node][value] = feat_val[0]
        else:
            tree[node][value] = buildtree(subtable,att_list.copy())

    return tree

In [26]:
att_list = list(df.keys()[:-1])
att_list

['age', 'income', 'student', 'credit rating']

In [27]:
att_list = list(df.keys()[:-1])
t = buildtree(df, att_list)

In [28]:
t

{'age': {'middle-aged': {'credit rating': {'excellent': {'student': {'no': 'no',
      'yes': {'income': {'high': 'no'}}}},
    'good': 'yes'}},
  'senior': {'credit rating': {'excellent': 'no',
    'good': {'student': {'no': 'no', 'yes': 'yes'}}}},
  'young': {'income': {'high': 'yes',
    'low': 'yes',
    'medium': {'student': {'no': {'credit rating': {'excellent': 'no',
        'good': 'yes'}},
      'yes': 'yes'}}}}}}